In [6]:
from __future__ import division, print_function
import numpy as np
from ase.build import bulk
from ase.calculators.lj import LennardJones
from ase.constraints import UnitCellFilter
from ase.optimize import MDMin
from ase.visualize import view
# Theoretical infinite-cutoff LJ FCC unit cell parameters
vol0 = 4 * 0.91615977036  # theoretical minimum
a0 = vol0**(1 / 3)

a = bulk('Al', 'fcc', a=a0)
cell0 = a.get_cell()

a.calc = LennardJones()
a.set_cell(np.dot(a.cell,
                  [[1.01, 0, 0],
                   [0, 1, 0],
                   [0, 0, 1]]),
           scale_atoms=True)

a *= (1, 2, 3)
cell0 *= np.array([1, 2, 3])[:, np.newaxis]

a.rattle()

# Verify analytical stress tensor against numerical value
s_analytical = a.get_stress()
s_numerical = a.calc.calculate_numerical_stress(a, 1e-5)
s_p_err = 100 * (s_numerical - s_analytical) / s_numerical

print("Analytical stress:\n", s_analytical)
print("Numerical stress:\n", s_numerical)
print("Percent error in stress:\n", s_p_err)
#assert np.all(abs(s_p_err) < 1e-5)

# Minimize unit cell
opt = MDMin(UnitCellFilter(a), dt=0.01)
opt.run(fmax=1e-3)

# Verify minimized unit cell using Niggli tensors
g_minimized = np.dot(a.cell, a.cell.T)
g_theory = np.dot(cell0, cell0.T)
g_p_err = 100 * (g_minimized - g_theory) / g_theory

print("Minimized Niggli tensor:\n", g_minimized)
print("Theoretical Niggli tensor:\n", g_theory)
print("Percent error in Niggli tensor:\n", g_p_err)
assert np.all(abs(g_p_err) < 1)

Analytical stress:
 [  3.59063244e-01  -9.07774787e-02  -9.20132530e-02   2.26366842e-04
  -4.50170945e-05  -3.17095857e-04]
Numerical stress:
 [  3.59063229e-01  -9.07774935e-02  -9.20132677e-02   2.26366875e-04
  -4.50170542e-05  -3.17095890e-04]
Percent error in stress:
 [ -4.20121096e-06   1.62982806e-05   1.60236328e-05   1.45779661e-05
  -8.94677171e-05   1.06263895e-05]
       Step     Time          Energy         fmax
MDMin:    0 13:59:32      -47.617795        0.4677
MDMin:    1 13:59:32      -47.617843        0.4577
MDMin:    2 13:59:33      -47.617981        0.4282
MDMin:    3 13:59:33      -47.618186        0.3819
MDMin:    4 13:59:33      -47.618433        0.3298
MDMin:    5 13:59:33      -47.618692        0.3281
MDMin:    6 13:59:33      -47.618944        0.3256
MDMin:    7 13:59:34      -47.619182        0.3224
MDMin:    8 13:59:34      -47.619408        0.3185
MDMin:    9 13:59:34      -47.619626        0.3138
MDMin:   10 13:59:34      -47.619843        0.3086
MDMin:   

In [7]:
1.195/0.01

119.5

In [2]:
from math import sqrt
from ase import Atoms
from ase.constraints import StrainFilter
from ase.optimize.mdmin import MDMin
from ase.io import Trajectory
try:
    from asap3 import EMT
except ImportError:
    pass
else:
    a = 3.6
    b = a / 2
    cu = Atoms('Cu', cell=[(0,b,b),(b,0,b),(b,b,0)], pbc=1) * (6, 6, 6)

    cu.set_calculator(EMT())
    f = StrainFilter(cu, [1, 1, 1, 0, 0, 0])
    opt = MDMin(f, dt=0.01)
    t = Trajectory('Cu.traj', 'w', cu)
    opt.attach(t)
    opt.run(0.001)

# HCP:
    from ase.build import hcp0001
    cu = hcp0001('Cu', (1, 1, 2), a=a / sqrt(2))
    cu.cell[1,0] += 0.05
    cu *= (6, 6, 3)

    cu.set_calculator(EMT())
    f = StrainFilter(cu)
    opt = MDMin(f, dt=0.01)
    t = Trajectory('Cu.traj', 'w', cu)
    opt.attach(t)
    opt.run(0.01)


In [2]:
import numpy as np

from ase import Atoms
from ase.io.trajectory import Trajectory
from ase.calculators.emt import EMT

a = 4.0  # approximate lattice constant
b = a / 2
ag = Atoms('Ag',
           cell=[(0, b, b), (b, 0, b), (b, b, 0)],
           pbc=1,
           calculator=EMT())  # use EMT potential
cell = ag.get_cell()
traj = Trajectory('Ag.traj', 'w')
for x in np.linspace(0.95, 1.05, 5):
    ag.set_cell(cell * x, scale_atoms=True)
    ag.get_potential_energy()
    traj.write(ag)

In [3]:
from ase.io import read
from ase.units import kJ
from ase.eos import EquationOfState
configs = read('Ag.traj@0:5')  # read 5 configurations
# Extract volumes and energies:
volumes = [ag.get_volume() for ag in configs]
energies = [ag.get_potential_energy() for ag in configs]
eos = EquationOfState(volumes, energies)
v0, e0, B = eos.fit()
print(B / kJ * 1.0e24, 'GPa')
eos.plot('Ag-eos.png')

100.14189241975146 GPa


In [3]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton
from ase.build import fcc111, add_adsorbate

h = 1.85
d = 1.10

slab = fcc111('Cu', size=(4, 4, 2), vacuum=10.0)

slab.set_calculator(EMT())
e_slab = slab.get_potential_energy()

molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])
molecule.set_calculator(EMT())
e_N2 = molecule.get_potential_energy()

add_adsorbate(slab, molecule, h, 'ontop')
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

print('Adsorption energy:', e_slab + e_N2 - slab.get_potential_energy())

                Step[ FC]     Time          Energy          fmax
*Force-consistent energies used in optimization.
BFGSLineSearch:    0[  0] 13:53:52       11.689927*       1.0797
BFGSLineSearch:    1[  2] 13:53:52       11.670814*       0.4090
BFGSLineSearch:    2[  4] 13:53:52       11.625880*       0.0409
Adsorption energy: 0.32351942231809083


In [4]:
slab.get_stress()

PropertyNotImplementedError: stress property not implemented